## Overview

This notebook allows to follow all the processing perfromed in the Behavioral_Analysis notebook ( which run behavioral_analysis functions)
It sometimes try to clarify some potential complications


In [1]:
import os
import glob
import pandas as pd
#import time


In [4]:
# the path_to_data_folder is the path of the folder where you store your different mice.

# Windows:
# path_to_data_folder='C:'+os.sep+'Users'+os.sep+'MORVAN'+os.sep+'Documents'+os.sep+'patchouris'+os.sep+'Sample_Data'+os.sep
# Linux:
# path_to_data_folder '/home/david/Documents/Code/ForagingProject/Patchouris/patchouris/Sample_Data/'
# MacOs
#path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'


#path_to_data_folder='/home/david/Documents/David/Data/'
path_to_data_folder='/LocalData/ForagingMice/JAK2Data'
path_to_data_folder='/LocalData/ForagingMice/MaudData'

#path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'


pattern_of_MOU_Folders = os.path.join(path_to_data_folder, "MOU*")

# List all mice in the data folder (If you want to process all the mice in your data folder),
mice_list = [os.path.basename(path) for path in glob.glob(pattern_of_MOU_Folders)]
mice_list=sorted(mice_list)

# Print the number of mice, the list of mice, and add an empty line
print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder: {", ".join(mice_list)}\n')


#If you want to process a subset of mice uncomment the line below and comment the 2 lines above

#mice_list: list[str] = ["MOU2334"]  # For processing  a single mice. Equivalent to mice_list = ["MOU2334"] but more correct as it forces to create a list of string
#mice_list=['MOU2329', 'MOU2330', 'MOU2331', 'MOU2332', 'MOU2333', 'MOU2334']
#print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder:')

session_list = {}
for mouse in mice_list:
    mouse_folder = os.path.join(path_to_data_folder,mouse)
    session_list[mouse] = sorted([name for name in os.listdir(mouse_folder)
                           if os.path.isdir(os.path.join(mouse_folder, name))
                           and name.startswith('MOU')])
    nb_sessions = len(session_list[mouse])
    print(f'Hello, I\'m {mouse}! I have foraged for {nb_sessions} sessions:')
    print(session_list[mouse], '\n')








Found 3 mice in the data folder: MOU4516, MOU4518, MOU4551

Hello, I'm MOU4516! I have foraged for 6 sessions:
['MOU4516_20240605-1103', 'MOU4516_20240605-1513', 'MOU4516_20240606-1144', 'MOU4516_20240606-1621', 'MOU4516_20240607-1101', 'MOU4516_20240607-1444'] 

Hello, I'm MOU4518! I have foraged for 6 sessions:
['MOU4518_20240605-1124', 'MOU4518_20240605-1533', 'MOU4518_20240606-1202', 'MOU4518_20240606-1638', 'MOU4518_20240607-1116', 'MOU4518_20240607-1500'] 

Hello, I'm MOU4551! I have foraged for 46 sessions:
['MOU4551_20240718-1049', 'MOU4551_20240718-1620', 'MOU4551_20240719-1047', 'MOU4551_20240719-1548', 'MOU4551_20240720-1016', 'MOU4551_20240720-1448', 'MOU4551_20240721-1015', 'MOU4551_20240721-1447', 'MOU4551_20240722-1010', 'MOU4551_20240722-1458', 'MOU4551_20240723-1013', 'MOU4551_20240723-1508', 'MOU4551_20240724-1013', 'MOU4551_20240724-1504', 'MOU4551_20240725-1015', 'MOU4551_20240725-1511', 'MOU4551_20240726-1004', 'MOU4551_20240726-1510', 'MOU4551_20240727-0955', 'MOU

We just print the list of mice, the dictionnany of sessions across mice and then we will choose a single mice and single session for illustration purpose

In [6]:
folder_path_mouse_to_process=os.path.join(path_to_data_folder,mice_list[2])
#folder_path_mouse_to_process=os.path.join(path_to_data_folder,'MOU4624')
print(folder_path_mouse_to_process)

session_to_process=session_list[mice_list[2]][0]
#session_to_process='MOU4624_20240806-1512'
print(session_to_process)


/LocalData/ForagingMice/MaudData/MOU4551
MOU4551_20240718-1049


In [22]:
import ast
def get_trapeze_and_tower_data(folder_path_mouse_to_process, session_to_process):

    """
    Function to extract trapeze width and tower coordinates from a session parameter CSV file.
    
    Parameters:
        folder_path_mouse_to_process (str): The folder path where the mouse data is stored.
        session_to_process (str): The specific session to process.

    Returns:
        trapeze_width (int or float): The width of the trapeze.
        towers_coordinates (dict): The coordinates of the towers.
    """
    # Load the session parameters CSV file
    param_file_path = os.path.join(folder_path_mouse_to_process, session_to_process, f"{session_to_process}_sessionparam.csv")
    param_df = pd.read_csv(param_file_path)

    # Check if the towers coordinates exist in the CSV file
    if "SE_coords" in param_df.columns:
        towers_coordinates = {
            "NW": param_df["NW_coords"].values[0],
            "NE": param_df["NE_coords"].values[0],
            "SW": param_df["SW_coords"].values[0],
            "SE": param_df["SE_coords"].values[0]
        }
        
        # Convert string representations of lists into actual lists
        towers_coordinates = {key: ast.literal_eval(value) for key, value in towers_coordinates.items()}

        print('Coordinates from parameter file:')
    else:
        # Default tower coordinates
        towers_coordinates = {
            "NW": [[104, 125], [173, 125], [173, 201], [104, 201]],
            "NE": [[330, 120], [400, 120], [400, 200], [330, 200]],
            "SW": [[109, 351], [181, 351], [181, 410], [109, 410]],
            "SE": [[330, 350], [400, 350], [400, 410], [330, 410]]
        }
        print('Default towers_coordinates')
    print(towers_coordinates)

    # Check if the trapeze size exists in the CSV file
    if "TrapezeSize" in param_df.columns:
        trapeze_width = param_df["TrapezeSize"].values[0]
        print('Trapeze width from parameter file:')
    else:
        trapeze_width = 50  # Default trapeze width
        print('Default trapeze width')
    print(trapeze_width)

    return trapeze_width, towers_coordinates


In [23]:
def generate_trapeze_and_tower_coordinates(towers_coordinates, trapeze_width):
    """
    Generates the coordinates of trapezes surrounding towers and converts all coordinates from pixels to centimeters.
    
    Parameters:
    towers_coordinates (dict): Dictionary containing the pixel coordinates of the 4 towers.
    trapeze_width (int): The width of the trapeze in pixels.
    
    
    Returns:
    tuple: 
        - all_trapezes_coordinates_cm (dict): Coordinates of the trapezes in cm.
        - towers_coordinates_cm (dict): Coordinates of the towers in cm.
    """
    
    #video_dimension_pixels (tuple): The resolution of the video in pixels 
    #arena_width_cm (float): The width of the arena in centimeters 
    #arena_width_pixels (int): The width of the arena in pixels 
    video_dimension_pixels=(512, 512)
    arena_width_cm=84
    arena_width_pixels=453



    # Conversion factor to go from pixel to cm
    conversion_factor = arena_width_cm / arena_width_pixels
    
    # Function to convert pixel coordinates to cm
    def convert_pix_to_cm(coordinate):
        return [round(coordinate[0] * conversion_factor, 2), round(coordinate[1] * conversion_factor, 2)]
    
    # Transform the coordinates to have the origin at the lower left (for plotting)
    max_y = video_dimension_pixels[1]
    transformed_towers_coordinates = {
        label: [[x, max_y - y] for x, y in vertices]
        for label, vertices in towers_coordinates.items()
    }
    
    # Function to generate trapeze coordinates surrounding a tower
    def trapeze_coordinates_from_tower(tower_coordinates, trapeze_width):
        trapeze_N = [
            tower_coordinates[0], tower_coordinates[1],
            [tower_coordinates[1][0] + trapeze_width, tower_coordinates[1][1] + trapeze_width],
            [tower_coordinates[0][0] - trapeze_width, tower_coordinates[0][1] + trapeze_width]
        ]
        trapeze_E = [
            tower_coordinates[1], tower_coordinates[2],
            [tower_coordinates[2][0] + trapeze_width, tower_coordinates[2][1] - trapeze_width],
            [tower_coordinates[1][0] + trapeze_width, tower_coordinates[1][1] + trapeze_width]
        ]
        trapeze_S = [
            tower_coordinates[2], tower_coordinates[3],
            [tower_coordinates[3][0] - trapeze_width, tower_coordinates[3][1] - trapeze_width],
            [tower_coordinates[2][0] + trapeze_width, tower_coordinates[2][1] - trapeze_width]
        ]
        trapeze_W = [
            tower_coordinates[3], tower_coordinates[0],
            [tower_coordinates[0][0] - trapeze_width, tower_coordinates[0][1] + trapeze_width],
            [tower_coordinates[3][0] - trapeze_width, tower_coordinates[3][1] - trapeze_width]
        ]
        return trapeze_N, trapeze_E, trapeze_S, trapeze_W
    
    # Initialize dictionaries to store trapeze and tower coordinates in cm
    all_trapezes_coordinates = {key: {} for key in towers_coordinates}
    all_trapezes_coordinates_cm = {}
    
    # Generate trapeze coordinates for each tower
    for tower_name, tower_coordinates in transformed_towers_coordinates.items():
        all_trapezes_coordinates[tower_name]["N"], \
        all_trapezes_coordinates[tower_name]["E"], \
        all_trapezes_coordinates[tower_name]["S"], \
        all_trapezes_coordinates[tower_name]["W"] = trapeze_coordinates_from_tower(tower_coordinates, trapeze_width)

    # Convert all trapeze coordinates from pixel to cm
    for tower, trapezes in all_trapezes_coordinates.items():
        all_trapezes_coordinates_cm[tower] = {
            trapeze: [convert_pix_to_cm(coord) for coord in coords]
            for trapeze, coords in trapezes.items()
        }
    
    # Convert tower coordinates from pixel to cm
    towers_coordinates_cm = {
        key: [convert_pix_to_cm(coord) for coord in transformed_towers_coordinates[key]]
    for key in transformed_towers_coordinates}

    return all_trapezes_coordinates_cm, towers_coordinates_cm


In [24]:
trapeze_width, towers_coordinates = get_trapeze_and_tower_data(folder_path_mouse_to_process, session_to_process)


Coordinates from parameter file:
{'NW': [[105, 120], [180, 120], [180, 195], [105, 195]], 'NE': [[325, 120], [405, 120], [405, 195], [325, 195]], 'SW': [[115, 342], [185, 342], [185, 412], [115, 412]], 'SE': [[325, 342], [400, 342], [400, 412], [325, 412]]}
Trapeze width from parameter file:
30


In [26]:
all_trapezes_coordinates_cm, towers_coordinates_cm= generate_trapeze_and_tower_coordinates(towers_coordinates, trapeze_width)
print(all_trapezes_coordinates_cm)

{'NW': {'N': [[19.47, 72.69], [33.38, 72.69], [np.float64(38.94), np.float64(78.25)], [np.float64(13.91), np.float64(78.25)]], 'E': [[33.38, 72.69], [33.38, 58.78], [np.float64(38.94), np.float64(53.22)], [np.float64(38.94), np.float64(78.25)]], 'S': [[33.38, 58.78], [19.47, 58.78], [np.float64(13.91), np.float64(53.22)], [np.float64(38.94), np.float64(53.22)]], 'W': [[19.47, 58.78], [19.47, 72.69], [np.float64(13.91), np.float64(78.25)], [np.float64(13.91), np.float64(53.22)]]}, 'NE': {'N': [[60.26, 72.69], [75.1, 72.69], [np.float64(80.66), np.float64(78.25)], [np.float64(54.7), np.float64(78.25)]], 'E': [[75.1, 72.69], [75.1, 58.78], [np.float64(80.66), np.float64(53.22)], [np.float64(80.66), np.float64(78.25)]], 'S': [[75.1, 58.78], [60.26, 58.78], [np.float64(54.7), np.float64(53.22)], [np.float64(80.66), np.float64(53.22)]], 'W': [[60.26, 58.78], [60.26, 72.69], [np.float64(54.7), np.float64(78.25)], [np.float64(54.7), np.float64(53.22)]]}, 'SW': {'N': [[21.32, 31.52], [34.3, 31.

### 3. this first part of the code generate the coordinates of the 4 towers and the trapezes around them

##### this step involves: 
##### 1- a tricky vertical symetries due to differential references of the coordinate entered in the aquisition program (ref on the upper left corner in opencv) and in matplotlib (ref on the lower left corner)
##### 2- a conversion from pixel to cm

In [ ]:
######################################
# maze information
######################################

# Mice have to run around 4 towers to obtain rewards. Rewards are delivered when a mouse switch from one trapeze to another concurrent trapeze



#        +-----------------+                            +-----------------+
#        | \  Trapeze N  / |                            | \  Trapeze N  / |
#        |  +-----------+  |                            |  +-----------+  |
#        |  |           |  |                            |  |           |  |
#        |TW|   Tower   |TE|                            |TW|   Tower   |TE|
#        |  |     NW    |  |                            |  |     NE    |  |
#        |  |           |  |                            |  |           |  |
#        |  +-----------+  |                            |  +-----------+  |
#        | /    Trap S   \ |                            | /    Trap S   \ |
#        +-----------------+                            +-----------------+





#        +-----------------+                            +-----------------+
#        | \  Trapeze N  / |                            | \  Trapeze N  / |
#        |  +-----------+  |                            |  +-----------+  |
#        |  |           |  |                            |  |           |  |
#        |TW|   Tower   |TE|                            |TW|   Tower   |TE|
#        |  |     SW    |  |                            |  |     SE    |  |
#        |  |           |  |                            |  |           |  |
#        |  +-----------+  |                            |  +-----------+  |
#        | /    Trap S   \ |                            | /    Trap S   \ |
#        +-----------------+                            +-----------------+


# We therefore need  the coordinate of the 4 towers and the trapeze. The coordinate are given in the reference frame of the video. 
# The video has a resolution of 512 pixeld in trapeze_width and in height. The point 0,0 is the left top corner.



towers_coordinates = {
    "NW": [[104, 125], [173, 125], [173, 201], [104, 201]],
    "NE": [[330, 120], [400, 120], [400, 200], [330, 200]],
    "SW": [[109, 351], [181, 351], [181, 410], [109, 410]],
    "SE": [[330, 350], [400, 350], [400, 410], [330, 410]]
}

trapeze_width = 50 # rewards are delivered in the maze around 4 towers,  when mice a switch from one trapze to another
#those are the coordinate taken directly from one of Maud acquisition code (june 2024) Line222

video_resolution = 512, 512 # this gives the resolution in pixel of the video recorded(trajectory of the mice in the maze)  
# Conversion factor to go from pixel to cm. We know that the width of the arena is 84 cm and the number of pixels is 453 
conversion_factor = 84 / 453
print(f"conversion factor cm per pixel { conversion_factor:.3f}")
# Function to convert pixels to cm
def convert_pix_to_cm(coordinate):
    return [round(coordinate[0] * conversion_factor, 2), round(coordinate[1] * conversion_factor, 2)]

# starting from July the towers coordinates are saved in the parameter csv file and mist be loaded .....




#if used as if the N and S are reversed. This is because openCV has the 0 in the upper left corner. 
# So these coordinate do not work in matplotlib environment where the origine is typically on the botoom left 




# This is the transformation we have to do to get the coordinates in pixel right, knowing that on open CV they go from 0,0 (upper left origin)  to (0,512), lower left max value
max_y = video_resolution[1]

# Transform the coordinates to have the origin at the lower left
transformed_towers_coordinates = {
    label: [[x, max_y - y] for x, y in vertices]
    for label, vertices in towers_coordinates.items()
}
print(transformed_towers_coordinates)
#Now let's draw the trapezes around the towers, knowing their width and the transformed_towers_coordinates




def trapeze_coordinates_from_tower(tower_coordinates, trapeze_width):
    """
    generate the trapezes coordinates surrounding a tower
    inputs:
    tower_coordinates - coordinates of the 4 corners of a tower [[Xa, Ya], [Xb, Yb], [Xc, Yc], [Xd, Yd]]
    trapeze_width - width of the trapeze in pixels
    outputs:
    coordinates [[Xa, Ya], [Xb, Yb], [Xc, Yc], [Xd, Yd]] for the 4 trapezes (N, E, S, W).
    
    
    """

    trapeze_N = [tower_coordinates[0], tower_coordinates[1], [tower_coordinates[1][0]+trapeze_width, tower_coordinates[1][1]+trapeze_width], [tower_coordinates[0][0]-trapeze_width, tower_coordinates[0][1]+trapeze_width]]
    trapeze_E = [tower_coordinates[1], tower_coordinates[2], [tower_coordinates[2][0]+trapeze_width, tower_coordinates[2][1]-trapeze_width], [tower_coordinates[1][0]+trapeze_width, tower_coordinates[1][1]+trapeze_width]]
    trapeze_S = [tower_coordinates[2], tower_coordinates[3], [tower_coordinates[3][0]-trapeze_width, tower_coordinates[3][1]-trapeze_width], [tower_coordinates[2][0]+trapeze_width, tower_coordinates[2][1]-trapeze_width]]
    trapeze_W = [tower_coordinates[3], tower_coordinates[0], [tower_coordinates[0][0]-trapeze_width, tower_coordinates[0][1]+trapeze_width], [tower_coordinates[3][0]-trapeze_width, tower_coordinates[3][1]-trapeze_width]]
    return trapeze_N, trapeze_E, trapeze_S, trapeze_W






# Initialize dictionary to store trapeze coordinates
all_trapezes_coordinates = {"NW": {}, "NE": {}, "SW": {}, "SE": {}}

# Iterate over each tower and compute trapeze coordinates
for tower_name, tower_coordinates in transformed_towers_coordinates.items():
    all_trapezes_coordinates[tower_name]["N"], \
    all_trapezes_coordinates[tower_name]["E"], \
    all_trapezes_coordinates[tower_name]["S"], \
    all_trapezes_coordinates[tower_name]["W"] = trapeze_coordinates_from_tower(tower_coordinates, trapeze_width)


print(all_trapezes_coordinates)




# Convert all coordinates from pixel to cm
all_trapezes_coordinates_cm = {}
for tower, trapezes in all_trapezes_coordinates.items():
    all_trapezes_coordinates_cm[tower] = {}
    for trapeze, coords in trapezes.items():
        all_trapezes_coordinates_cm[tower][trapeze] = [convert_pix_to_cm(coord) for coord in coords]

# Print the result for the trapezes
for tower, trapezes in all_trapezes_coordinates_cm.items():
    print(f"{tower}:")
    for trapeze, coords in trapezes.items():
        print(f"  {trapeze}: {coords}")
# Create a new dictionary with coordinates converted to cm
towers_coordinates_cm = {}

for key in transformed_towers_coordinates:
    towers_coordinates_cm[key] = [convert_pix_to_cm(coord) for coord in transformed_towers_coordinates[key]]

print(towers_coordinates_cm)

In [ ]:
all_trapezes_coordinates_cm

In [ ]:
towers_coordinates_cm